## Repliikide lausestamine

Eelnevalt segmenteeriti transkriptsioonifailid nimeüksusi sisaldavateks repliikideks. Repliigid võivad aga olla kohati üpris pikad tekstilõigud, mille töötlemine võib osutuda hiljem keerukaks. Seetõttu segmenteeritakse need siin hilisema töö lihtsustamise huvides lauseteks.

In [2]:
# Vajalikud impordid
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from estnltk_core.layer_operations import split_by_sentences

### I Andmete sisselugemine

Esmalt loetakse andmebaasist sisse sinna varem salvestatud repliigid.

Tühi järjend repliikide hoiustamiseks:

In [3]:
corpus = []

Tühi järjend repliikide ID-de jaoks:

In [4]:
line_ids = []

Andmebaasi tabelist loetakse sisse repliikidele vastavad Text-objektid:

In [5]:
con = sqlite3.connect("media_data_complete.db")

In [6]:
cur = con.cursor()

In [7]:
for row in cur.execute("SELECT ID, line FROM lines"):
    line_id = row[0]
    line = json_to_text(json_text=row[1])
    corpus.append(line)
    line_ids.append(line_id)

In [8]:
con.close()

In [9]:
len(corpus)

34812

Vaatame üht repliiki:

In [10]:
test_text = corpus[100]
display(test_text)

Text(text='Sudaanis meediaväljaandeid ja neid kes end uuringutega tegelevad üles mingisugusele koordineerimisele.\nEt mitte samal päeval ajavalt erineva metoodikaga tehtud küsitlusi näiteks et tingimata väga selgelt ja arusaadavalt välja tuua mis Birma poolt tehtud see on ja mida konkreetset küsiti et see oleks nagu peale ja sealt on siis Emori uuringu turuuuringuid uute riikide enese posti Marvi.')

Ja selles repliigis esinevaid nimeüksusi:

In [11]:
test_text.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Sudaanis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Birma'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Emori'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Marvi'], [{'nertag': 'PER'}])])

### II Repliikide lausestamine

Repliikide lausestamiseks kasutatakse siin funktsiooni split_by_sentences, mis lööb teksti lauseteks lauselõpumärkide kohalt, säilitades samal ajal juba olemasolevad märgenduskihid. Küll aga ei lausesta see teksti, kus lauselõpumärgid puuduvad (mida võib tulla ette automaatselt transkribeeritud failide või muu mitte päris kirjakeelse teksti puhul). Seetõttu jäävad sellised repliigid oma esialgsele kujule, kuigi need salvestatakse lihtsuse mõttes hiljem koos lausetega samasse tabelisse.

In [12]:
line_sentences = split_by_sentences(text=test_text,
                           layers_to_keep=list(test_text.layers),
                           trim_overlapping=True
                           )

Näiteks esimene lause näidisrepliigist:

In [13]:
line_sentences[0]

Text(text='Sudaanis meediaväljaandeid ja neid kes end uuringutega tegelevad üles mingisugusele koordineerimisele.')

### III Lausete salvestamine andmebaasi

In [20]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

Andmebaasi tekitatakse uus tabel lausete jaoks:

In [21]:
cur.execute("CREATE TABLE sentences(ID INTEGER PRIMARY KEY, line_ID INTEGER, sentence TEXT, named_entity INTEGER)")

Kõik repliigid lausestatakse ja saadud laused lisatakse loodud tabelisse. Eraldi märge lisatakse veergu named_entity, kas lause sialdab mõnda NerTaggeri abil leitud nimeüksust või mitte:

In [22]:
for i in range(len(corpus)):
    line_sentences = split_by_sentences(text=corpus[i], 
                           layers_to_keep=list(corpus[i].layers),
                           trim_overlapping=True
                           )
    for sentence in line_sentences:
        # igaks juhuks lisatakse lausetele ka metaandmed
        sentence.meta['line_number'] = corpus[i].meta['line_number']
        sentence.meta['person'] = corpus[i].meta['person']
        sentence.meta['date'] = corpus[i].meta['date']
        sentence_json = text_to_json(sentence)
        if len(sentence.ner) > 0:
            cur.execute("""INSERT INTO sentences
                                    (line_ID, sentence, named_entity)
                                    VALUES (?, ?, ?);""", (line_ids[i], sentence_json, 1))
        else:
            cur.execute("""INSERT INTO sentences
                                    (line_ID, sentence, named_entity)
                                    VALUES (?, ?, ?);""", (line_ids[i], sentence_json, 0))
        con.commit()

In [23]:
con.close()